In [79]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append("../")

location = "remote"
if location == "remote":
    # TODO: hacky, shouldn't be necessary
    os.environ["WANDB_NOTEBOOK_NAME"] = "lustre_scratch/coralshift/notebooks/rnn.ipynb"
    os.chdir("/lustre_scratch/orlando-code/coralshift/")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
from __future__ import annotations

from pathlib import Path
import xarray as xa
import numpy as np
import math as m
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import wandb
from tqdm import tqdm
from sklearn import model_selection
from sklearn.preprocessing import normalize
from scipy.interpolate import interp2d
from sklearn.utils import class_weight

import xbatcher

import rasterio
from rasterio.plot import show
import rioxarray as rio

from bs4 import BeautifulSoup
import requests


#issues with numpy deprecation in pytorch_env
from coralshift.processing import spatial_data
from coralshift.utils import file_ops, directories
from coralshift.plotting import spatial_plots, model_results
from coralshift.dataloading import data_structure, climate_data

In [90]:
os.chdir(os.path.expanduser("~"))

# out = xa.open_dataset("lustre_scratch/datasets/1993010102201/1-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA11_G_1993001_night-v02.0-fv01.0.nc")
# out

In [31]:
out.sel({"lat":slice(-10,-17)})

<xarray.Dataset>
Dimensions:                    (time: 1, nv: 2, lat: 168, lon: 8640)
Coordinates:
  * time                       (time) datetime64[ns] 1992-12-31T22:02:56
  * lat                        (lat) float32 -10.02 -10.06 ... -16.94 -16.98
  * lon                        (lon) float32 -180.0 -179.9 ... 179.9 180.0
Dimensions without coordinates: nv
Data variables: (12/15)
    time_bounds                (time, nv) datetime64[ns] ...
    lat_bounds                 (lat, nv) float32 ...
    lon_bounds                 (lon, nv) float32 ...
    crs                        int32 ...
    sea_surface_temperature    (time, lat, lon) float32 ...
    sst_dtime                  (time, lat, lon) timedelta64[ns] ...
    ...                         ...
    wind_speed                 (time, lat, lon) float32 ...
    sea_ice_fraction           (time, lat, lon) float32 ...
    aerosol_dynamic_indicator  (time, lat, lon) float32 ...
    quality_level              (time, lat, lon) float32 ...
    pathfinder_quality_level   (time, lat, lon) float32 ...
    l2p_flags                  (time, lat, lon) float64 ...
Attributes: (12/74)
    Conventions:                             CF-1.6, ACDD-1.3
    title:                                   AVHRR Pathfinder Version 5.3 L3-...
    summary:                                 This netCDF-4 file contains sea ...
    references:                              http://pathfinder.nodc.noaa.gov ...
    institution:                             NCEI
    history:                                 smigen_both ifile=1993001.b4kd1-...
    ...                                      ...
    platform_vocabulary:                     NASA Global Change Master Direct...
    instrument_vocabulary:                   NASA Global Change Master Direct...
    metadata_link:                           http://data.nodc.noaa.gov/cgi-bi...
    program:                                 NOAA Climate Data Record (CDR) P...
    cdr_program:                             NOAA Climate Data Record Program...
    cdr_id:                                  gov.noaa.ncdc:C00983

In [57]:
out = process_xa_d(out)

In [63]:
xa_region_from_coord_bounds(out, {"latitude": (-17, -12), "longitude": (-142, -10)})


<xarray.Dataset>
Dimensions:                    (time: 1, nv: 2, latitude: 120, longitude: 3168)
Coordinates:
  * time                       (time) datetime64[ns] 1992-12-31T22:02:56
  * latitude                   (latitude) float32 -16.98 -16.94 ... -12.02
  * longitude                  (longitude) float32 -142.0 -141.9 ... -10.02
Dimensions without coordinates: nv
Data variables: (12/15)
    time_bounds                (time, nv) datetime64[ns] ...
    lat_bounds                 (latitude, nv) float32 ...
    lon_bounds                 (longitude, nv) float32 ...
    crs                        int32 ...
    sea_surface_temperature    (time, latitude, longitude) float32 ...
    sst_dtime                  (time, latitude, longitude) timedelta64[ns] ...
    ...                         ...
    wind_speed                 (time, latitude, longitude) float32 ...
    sea_ice_fraction           (time, latitude, longitude) float32 ...
    aerosol_dynamic_indicator  (time, latitude, longitude) float32 ...
    quality_level              (time, latitude, longitude) float32 ...
    pathfinder_quality_level   (time, latitude, longitude) float32 ...
    l2p_flags                  (time, latitude, longitude) float64 ...
Attributes: (12/74)
    Conventions:                             CF-1.6, ACDD-1.3
    title:                                   AVHRR Pathfinder Version 5.3 L3-...
    summary:                                 This netCDF-4 file contains sea ...
    references:                              http://pathfinder.nodc.noaa.gov ...
    institution:                             NCEI
    history:                                 smigen_both ifile=1993001.b4kd1-...
    ...                                      ...
    platform_vocabulary:                     NASA Global Change Master Direct...
    instrument_vocabulary:                   NASA Global Change Master Direct...
    metadata_link:                           http://data.nodc.noaa.gov/cgi-bi...
    program:                                 NOAA Climate Data Record (CDR) P...
    cdr_program:                             NOAA Climate Data Record Program...
    cdr_id:                                  gov.noaa.ncdc:C00983

In [86]:
def process_xa_d(xa_d: xa.Dataset | xa.DataArray):
    # standardise coordinate names
    temp_xa_d = xa_d.rename({"lat": "latitude", "lon": "longitude"})
    # order variables
    return temp_xa_d.sortby(list(temp_xa_d.coords))

def xa_region_from_coord_bounds(xa_d, coord_bounds_dict):
    slice_dict = {}
    for k, v in coord_bounds_dict.items():
        slice_dict[k] = slice(min(v), max(v))

    return xa_d.sel(slice_dict)

def get_links_from_url(url, suffix: str=".nc"):
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    hrefs = soup.find_all("a", href=lambda href: href.endswith(".nc"))
    links = ["/".join((url, hrefs["href"])) for hrefs in hrefs]
    return links


def fetch_pathfinder_year_urls(year):
    url_root = "https://www.ncei.noaa.gov/data/oceans/pathfinder/Version5.3/L3C/"
    url_root_year_data = "/".join((str(url_root), str(year), "data"))
    return get_links_from_url(url_root_year_data, ".nc")


def download_save_url_file_to_path(url, save_path):
    r = requests.get(url)
    with open(save_path, "wb") as f:
        f.write(r.content)

def download_and_sample_pathfinder(download_dest_dir, years: list[int], coord_bounds_dict):
    for year in years:
        # scrape filepaths from url
        nc_urls = fetch_pathfinder_year_urls(year)

        for url in tqdm(nc_urls, desc = f"Processing urls for {year}"):
            save_path = Path(download_dest_dir) / Path(url).name
            # download to specified path
            # download_save_url_file_to_path(url, save_path)
            check_exists_download_url(save_path, url)
            # open file and extract region
            xa_d = xa.open_dataset(save_path, chunks={"time": 100, "latitude": 10, "longitude": 10})
            xa_d_subregion = xa_region_from_coord_bounds(process_xa_d(xa_d), coord_bounds_dict)
            # save relevant area to .nc with same filepath (overwrite current file to save space)
            # TODO: overcome file permissions
            xa_d_subregion.to_netcdf(str(save_path))
    print(f"All urls downloaded and limited in extent: {str(coord_bounds_dict)}.")

In [85]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path, loading_bar: bool = True) -> None:
    print("\n")
    with DownloadProgressBar(
        unit="B", unit_scale=True, miniters=1, desc=url.split("/")[-1]
    ) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)
    print(f"Download to {output_path} complete.")


def check_exists_download_url(
    filepath: Path | str, url: str, loading_bar: bool = True
) -> None:
    """Download a file from a URL to a given filepath, with the option to display a loading bar.

    Parameters
    ----------
        filepath (Path | str): future path at which the downloaded file will be stored
        url (str): URL from which to download the file
        loading_bar (bool, optional): Whether to display a loading bar to show download progress. Defaults to True.

    Returns
    -------
        None
    """
    # if not downloaded
    if not Path(filepath).is_file():
        # download with loading bar
        if loading_bar:
            download_url(url, str(filepath))
        # choose to download without loading bar
        else:
            urllib.request.urlretrieve(url, filename=filepath)
    # if already downloaded
    else:
        print(f"Already exists: {filepath}")

In [88]:
import urllib

In [91]:
download_and_sample_pathfinder(
    "lustre_scratch/datasets/pathfinder_sst", [1993], {"latitude": (-17, -12), "longitude": (-142, -10)}
)

Processing urls for 1993:   0%|          | 0/730 [00:00<?, ?it/s]

19930101022011-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA11_G_1993001_night-v02.0-fv01.0.nc: 38.5MB [00:02, 13.1MB/s]                            


Download to lustre_scratch/datasets/pathfinder_sst/19930101022011-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA11_G_1993001_night-v02.0-fv01.0.nc complete.


Processing urls for 1993:   0%|          | 0/730 [00:06<?, ?it/s]


PermissionError: [Errno 13] Permission denied: '/home/jovyan/lustre_scratch/datasets/pathfinder_sst/19930101022011-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA11_G_1993001_night-v02.0-fv01.0.nc'

## Load in data

In [3]:
ds_man = data_structure.MyDatasets()
ds_man.set_location(location)

noaa_features = ['mlotst', 'bottomT', 'uo', 'so', 'zos', 'thetao', 'vo']

# TODO: transparency in preprocessing to get to this (probably split into separate gt datarray)
ds_man.add_dataset(
    "monthly_climate_1_12", xa.open_dataset(
        ds_man.get_location() / "global_ocean_reanalysis/monthly_means/coral_climate_1_12.nc")
)

ds_man.add_dataset(
    "monthly_climate_features", ds_man.get_dataset("monthly_climate_1_12")[noaa_features]
)

# ds_man.add_dataset(
#     "monthly_climate_1_12_X_y_np", spatial_data.filter_out_nans(
#         spatial_data.xa_ds_to_3d_numpy(ds_man.get_dataset("monthly_climate_1_12")), 
#         np.array(ds_man.get_dataset("monthly_climate_1_12")["coral_algae_1-12_degree"].isel(time=-1)).reshape(-1, 1))
# )
ds_man.add_datasets(
    ["monthly_climate_1_12_X", "monthly_climate_1_12_y"], 
        spatial_data.process_xa_ds_for_ml(ds_man.get_dataset("monthly_climate_1_12"), 
        feature_vars=noaa_features, gt_var="coral_algae_1-12_degree")
)

# TODO: handle depth
ds_man.add_dataset(
    "daily_climate_1_12", spatial_data.generate_and_add_gt_to_xa_d(xa.open_dataset(
        Path(ds_man.get_location() / "global_ocean_reanalysis/daily_means/dailies_combined.nc")).isel(depth=0),
        ds_man.get_dataset("monthly_climate_1_12")["coral_algae_1-12_degree"])
)

# TODO: streamline checking and saving process
daily_climate_1_12_X_file_path = ds_man.get_location() / "global_ocean_reanalysis/daily_means/daily_climate_1_12_X.npy"
# if daily_climate_1_12_X numpy array doesn't exist, generate and save
if not file_ops.check_file_exists(filepath = daily_climate_1_12_X_file_path):
    daily_climate_1_12_X = spatial_data.process_xa_ds_for_ml(ds_man.get_dataset("daily_climate_1_12"),
        feature_vars = noaa_features)
    np.save(daily_climate_1_12_X_file_path, daily_climate_1_12_X) 
    ds_man.add_dataset("daily_climate_1_12_X", np.load(daily_climate_1_12_X_file_path))
else:
    ds_man.add_dataset("daily_climate_1_12_X", np.load(daily_climate_1_12_X_file_path))

daily_climate_1_12_padded_1_file_path = ds_man.get_location() / "global_ocean_reanalysis/daily_means/daily_climate_1_12_padded_1.nc"
# if daily_climate_1_12_padded_1 .nc file doesn't exist, generate and save
if not file_ops.check_file_exists(filepath = daily_climate_1_12_padded_1_file_path):
    daily_climate_1_12_padded_1 = spatial_data.spatially_buffer_timeseries(
        ds_man.get_dataset("daily_climate_1_12"), buffer_size=1, exclude_vars = ["spatial_ref", "coral_algae_gt"])
    daily_climate_1_12_padded_1.to_netcdf(filepath = daily_climate_1_12_padded_1_file_path)
    ds_man.add_dataset("daily_climate_1_12_padded_1", xa.open_dataset(daily_climate_1_12_padded_1_file_path))
else:
    ds_man.add_dataset("daily_climate_1_12_padded_1", xa.open_dataset(daily_climate_1_12_padded_1_file_path))

# add in ground truth to padded
ds_man.add_dataset(
    "daily_climate_1_12_padded_1_gt", spatial_data.generate_and_add_gt_to_xa_d(
        ds_man.get_dataset("daily_climate_1_12_padded_1"),
        ds_man.get_dataset("monthly_climate_1_12")["coral_algae_1-12_degree"])
)

ds_man.add_dataset(
    "bathymetry_A", rio.open_rasterio(
        rasterio.open(ds_man.get_location() / "bathymetry/GBR_30m/Great_Barrier_Reef_A_2020_30m_MSL_cog.tif"),
        ).rename("bathymetry_A").rename({"x": "longitude", "y": "latitude"})
)

## GRU function definitions 

In [ ]:
xa_coral_climate_1_12_features = ds_man.get_dataset("monthly_climate_features")
xa_coral_climate_1_12 = ds_man.get_dataset("monthly_climate_1_12")

xa_coral_climate_1_12_working = xa_coral_climate_1_12

In [ ]:
# all_Xs_onehot, all_lat_lon_dict_onehot = sample_spatial_batch(xa_coral_climate_1_12, lat_lon_starts=(-8,140), coord_range=(-20,13))
# all_Xs_onehot, all_lat_lon_dict_onehot = subsample_to_array(xa_coral_climate_1_12, lat_lon_starts=(-8,140), coord_range=((-20,13)))
# all_Xs_onehot = naive_X_nan_replacement(all_Xs_onehot)
# all_ys_onehot, _ = subsample_to_array(xa_coral_climate_1_12, lat_lon_starts=(-8,140), coord_range=(-20,13), variables = ["coral_algae_1-12_degree"])
# all_ys_onehot = naive_y_nan_replacement(all_ys_onehot)
# all_ys_onehot = all_ys_onehot[:,:,0]

train_onehot_Xs, train_onehot_ys, train_onehot_subsample, train_onehot_lat_lons_vals_dict = generate_patch(xa_ds=xa_coral_climate_1_12, lat_lon_starts=(-10,142), coord_range=(-6,6), onehot=False)
test_onehot_Xs, test_onehot_ys, test_onehot_subsample, test_onehot_lat_lons_vals_dict = generate_patch(xa_ds=xa_coral_climate_1_12, lat_lon_starts=(-16,148), coord_range=(-6,6))

print("train_onehot_Xs shape: ", train_onehot_Xs.shape)
print("train_onehot_ys shape: ", train_onehot_ys.shape)

In [ ]:
# load bathymetry
bath_A = ds_man.get_dataset("bathymetry_A")
bath_A

# 1 km. Struggles displaying/processing 100m, but have yet to try saving to this/inferring
target_resolution = 1000
_,_,av_degrees = spatial_data.distance_to_degrees(target_resolution)
bath_A_1km = spatial_data.upsample_xarray_to_target(bath_A, av_degrees)
# im = bath_A_1km.plot(ax=ax)

spatial_plots.plot_DEM(bath_A_1km, f" DEM upsampled to {target_resolution} meters", vmin=-100, vmax=0)
# spatial_plots.format_spatial_plot(im, fig, ax, f"Upsampled to {target_resolution} degrees")

In [ ]:
def find_chunks_with_percentage(
    array: np.ndarray,
    range_min: float,
    range_max: float,
    chunk_size: int,
    threshold_percent: float,
    preprocess: bool = True
) -> list[tuple[float, float]]:
    """Find chunks in the array that contain a certain threshold percentage of pixel values within a specified range.

    Parameters
        array (ndarray): Input array.
        range_min (float): Minimum value for the range.
        range_max (float): Maximum value for the range.
        chunk_size (int): Size of the chunks in rows and columns.
        threshold_percent (float): Threshold percentage for chunk selection.

    Returns:
        tuple[list[tuple[float, float]], list[float]]: List of chunk coordinates that meet the threshold percentage
            criteria and list of percentage of grid cells meeting criteria.
    """
    if preprocess:
        array = preprocess_array(array, range_min, range_max)

    # to make chunk_size behave as expected in the face of non-inclusive final indices
    chunk_size = chunk_size + 1
    rows, cols = array.shape
    chunk_rows = np.arange(0, rows - chunk_size, chunk_size)
    chunk_cols = np.arange(0, cols - chunk_size, chunk_size)

    chunk_coords = []
    cell_coverages = []

    for start_row in tqdm(chunk_rows, desc="Calculating areas within value range"):
        for start_col in chunk_cols:
            # Calculate the sub-array within the chunk
            sub_array = array[start_row:start_row + chunk_size, start_col:start_col + chunk_size]

            # Count the number of values within the range
            count = np.sum((sub_array >= range_min) & (sub_array <= range_max))

            # Calculate the coverage percentage
            cell_coverage = (count / (chunk_size ** 2)) * 100

            if cell_coverage >= threshold_percent:
                chunk_coords.append(
                    (
                        (start_row, start_col),
                        (start_row + chunk_size - 1, start_col + chunk_size - 1),
                    )
                )
                cell_coverages.append(cell_coverage.item())

    return chunk_coords, cell_coverages


def preprocess_array(array: np.ndarray, range_min: float, range_max: float) -> np.ndarray:
    """
    Preprocess the input array by filtering out values outside the desired range.

    Parameters:
        array (ndarray): Input array.
        range_min (float): Minimum value for the desired range.
        range_max (float): Maximum value for the desired range.

    Returns:
        ndarray: Preprocessed array with values filtered within the desired range.
    """
    return np.where(np.logical_and(array >= range_min, array <= range_max), array, np.nan)



def nc_chunk_files(
    dest_dir_path: Path | str,
    xa_ds: xa.Dataset,
    chunk_size: int = 20,
    threshold_percent: float = 10,
    vmin: float = -100,
    vmax: float = 0,
):
    """Save chunks of an xarray Dataset to NetCDF files along with accompanying metadata JSON files.

    Parameters
    ----------
    dest_dir_path (Path | str): Directory path to save the chunk files.
    xa_ds (xa.Dataset): Input xarray Dataset.
    chunk_size (int, optional): Size of the chunks (default is 20).
    threshold_percent (float, optional): Threshold percentage for chunk coverage (default is 10).
    vmin (float, optional): Minimum value for chunk selection (default is -100).
    vmax (float, optional): Maximum value for chunk selection (default is 0).

    Returns
    -------
    None
    """

    chunk_coord_pairs, coverages = find_chunks_with_percentage(
        xa_ds.values, vmin, vmax, chunk_size, threshold_percent
    )

    for i, coord_pair in tqdm(
        enumerate(chunk_coord_pairs),
        desc="Saving chunks .nc and accompanying metadata .json files",
        total=len(chunk_coord_pairs),
    ):
        sub_ds = spatial_data.ds_subsample_from_coord(xa_ds, coord_pair)
        # generate filename and file_path
        filename = "_".join(
            ("chunk", utils.pad_number_with_zeros(number=i, resulting_len=3))
        )
        file_path = Path(dest_dir_path) / filename
        # generate chunk metadata
        info_dict = generate_chunk_json(sub_ds, file_path, coord_pair, coverages[i])
        # save metadata file
        file_ops.save_json(
            info_dict, filepath=file_path.with_suffix(".json"), verbose=False
        )
        # save nc file
        sub_ds.to_netcdf(path=file_path.with_suffix(".nc"))

    print(f".nc chunk files and accompanying metadata written to {str(dest_dir_path)}")
    return chunk_coord_pairs, coverages


def add_data_to_chunk(xa_chunk_data, xa_new_data, interp_method: str="nearest", broadcast: bool=True):
    # TODO: add in checking about relative resolutions
    interped = xa_new_data.interp(
        latitude=xa_chunk_data["latitude"], longitude=xa_chunk_data["longitude"], method=interp_method)
    # TODO: issue with time broadcasting?
    merged_ds = xa.combine_by_coords([xa_chunk_data, interped], coords=['latitude', 'longitude', 'time'], join="inner")
    if broadcast:
        (merged_ds,) = xa.broadcast(merged_ds)
    return merged_ds
    # save file


def add_data_to_chunks(chunk_dir: Path | str, xa_new_data_area, new_dir_name: str):
    # determine chunk_paths folder
    chunk_paths = file_ops.return_list_filepaths(chunk_dir, ".nc", incl_subdirs=False)
    # make new folder for writing merged data to
    new_dir = file_ops.guarantee_existence(Path(chunk_dir) / new_dir_name)

    for p in tqdm(chunk_paths, desc="Merging additional data into .nc chunk files"):
        p = Path(p)
        chunk_file_name = p.stem
        new_chunk_path = new_dir / p.stem
        xa_chunk_data = xa.open_dataset(p)
        # determine spatial limts TODO: could replace this with reading from metadata (potentially faster)
        lat_lims = spatial_data.xarray_coord_limits(xa_chunk_data, "latitude") 
        lon_lims = spatial_data.xarray_coord_limits(xa_chunk_data, "longitude")
        # TODO: make selection of index order more universal (currently only good for negative lats)
        extra_xa_chunk_data = xa_new_data_area.sel({
            "latitude": slice(lat_lims[0], lat_lims[1]),
            "longitude": slice(lon_lims[0], lon_lims[1])
            })

        # determine data to add TODO: change
        merged_ds = add_data_to_chunk(xa_chunk_data, extra_xa_chunk_data)
        info_dict = generate_chunk_json(merged_ds, new_chunk_path)
        file_ops.save_json(info_dict, filepath=new_chunk_path.with_suffix(".json"), verbose=False)
        merged_ds.to_netcdf(path=new_chunk_path.with_suffix(".nc"))
    print(f"Merged chunks written to .nc files at {str(new_dir)}")

def generate_chunk_json(
    xa_d: xa.DataArray,
    file_path: str | Path,
    coord_pair: tuple[int] = None,
    coverage: float = None,
) -> dict:
    """Generate metadata JSON dictionary for a chunk of an xarray DataArray.

    Parameters
    ----------
    xa_d (xa.DataArray): Input xarray DataArray.
    file_path (str | Path): File path of the chunk.
    coord_pair (tuple[int]): Index pair containing the start and end coordinates.
    coverage (float): Chunk coverage value.

    Returns
    -------
    dict: Metadata JSON dictionary.
    """

    # TODO: make robust with different dataset/dataarray

    # make filename
    vars = spatial_data.get_vars_from_ds_or_da(xa_d)
    # convert coord indices to absolute coords (TODO: could add spatial functionality for chunking)
    lat_lims = spatial_data.xarray_coord_limits(xa_d, "latitude")
    lon_lims = spatial_data.xarray_coord_limits(xa_d, "longitude")
    # find resolution
    lat_resolution_d, lon_resolution_d = spatial_data.calculate_spatial_resolution(xa_d)
    lat_resolution_m, lon_resolution_m = spatial_data.degrees_to_distances(
        lat_resolution_d, lon_resolution_d
    )


    info_dict = {
        "file name": file_path.stem,
        "file path": str(file_path),
        "variables": vars,
        "latitude range": lat_lims,
        "longitude range": lon_lims,
        "latitude resolution (degrees)": lat_resolution_d,
        "longitude resolution (degrees)": lon_resolution_d,
        "latitude resolution (meters)": lat_resolution_m,
        "longitude resolution (meters)": lon_resolution_m,
    }

    if coord_pair is not None:
        # calculate minimum and maximum bathymetries
        min_bath, max_bath = xa_d.values.min(), xa_d.values.max()
        additional_info = {
            "latitude chunk size": np.diff((coord_pair[0][0], coord_pair[1][0])).item(),
            "longitude chunk size": np.diff(
                (coord_pair[0][1], coord_pair[1][1])
            ).item(),
            "start index pair": coord_pair[0],
            "end index pair": coord_pair[1],
            "minimum bathymetry": min_bath,
            "maximum bathymetry": max_bath,
        }
        info_dict.update(additional_info)
    if coverage is not None:
        additional_info = {
            "cell coverage": coverage,
        }
        info_dict.update(additional_info)

    return info_dict

In [ ]:
from scipy.ndimage import generic_filter


# assign nans to zero
def calculate_slopes(array: np.ndarray, buffer_size: int = 3) -> np.ndarray:

    def slope_sweeper(values:np.ndarray):
        # bathymetry cell of interest
        central_value = values[len(values) // 2]
        # replace any nan values (outside array or coast) with bathymetry cell of interest (approximation)
        values[np.isnan(values)] = central_value
        # remove the duplicated central value
        # contig_vals = remove_single_instance(values, central_value)
        contig_vals = values[1::2]  # TODO: generalise for non-3kernel sizes
        diff = abs(((np.sum(contig_vals)) - central_value * len(contig_vals))/len(contig_vals))
        return diff
        
    sloped_array = generic_filter(array, slope_sweeper, size=buffer_size, mode="constant", cval=np.nan)

    return sloped_array


def calculate_cell_slopes(bath: xa.DataArray, buffer_size: int = 3):
    slopes = xa.apply_ufunc(
        calculate_slopes,
        bath,
        input_core_dims=[[]],
        output_core_dims=[[]],
        kwargs={"buffer_size": buffer_size},
    )
    return slopes


def remove_single_instance(array, value):
    index = np.where(array == value)[0][0]
    return np.concatenate((array[:index], array[index+1:]))

# slopes = calculate_cell_slopes(test_array, buffer_size=3)
slopes = calculate_cell_slopes(bath_A_1km.values[0,:,:], buffer_size=3)

In [ ]:
test_array = np.arange(0,16,1).reshape(4,4)

In [ ]:
f, a = plt.subplots()
im = a.imshow(bath_A_1km.values[0,:,:], vmin=-100, vmax=0)
f.colorbar(im, ax=a)

In [ ]:
from matplotlib import colors

slopes = gaussian_gradient_magnitude(bath_A_1km.values[0,:,:], sigma=1)

row_start, row_end = 200, 300
col_start, col_end = 180, 260

bath_vals = bath_A_1km.values[0, row_start:row_end, col_start:col_end]
slope_vals = slopes[row_start:row_end, col_start:col_end]
# bath_vals[bath_vals < -500] = -500

if bath_vals.max() < 0:
    divnorm_bath = colors.TwoSlopeNorm(vmin=bath_vals.min(), vcenter=0, vmax=0.1)
else:
    divnorm_bath = colors.TwoSlopeNorm(vmin=bath_vals.min(), vcenter=0., vmax=bath_vals.max())
# if slope_vals.max() < 0:
#     divnorm_slope = colors.TwoSlopeNorm(vmin=slope_vals.min(), vcenter=slope_vals.max(), vmax=0)
# else:
#     divnorm_slope = colors.TwoSlopeNorm(vmin=slope_vals.min(), vcenter=0., vmax=0.1)


f, ax = plt.subplots(1,2, figsize=[10,5])
bath_im = ax[0].imshow(bath_vals, cmap="RdPu_r", vmin=bath_vals.min(), vmax=bath_vals.max()
    # norm=divnorm_bath
    )
ax[0].set_title("bathymetry")
f.colorbar(bath_im, ax=ax[0])

slope_im = ax[1].imshow(slope_vals, cmap="RdPu",
    # norm=divnorm_slope
    )
ax[1].set_title("absolute slope value")
f.colorbar(slope_im, ax=ax[1])


In [ ]:
from scipy.ndimage import gaussian_gradient_magnitude

plt.imshow(gaussian_gradient_magnitude(bath_A_1km.values[0,:,:], sigma=1))

In [ ]:
# Create a 100x100 array of zeros
arr = np.zeros((5, 5))

# Set the central square of size 10x10 to 100
start = 1
end = 4
arr[start:end, start:end] = 100

In [ ]:
slopes = calculate_cell_slopes(arr, buffer_size=3)

f, ax = plt.subplots(1,2, figsize=[10,5])
bath = ax[0].imshow(arr)
ax[0].set_title("bathymetry")
f.colorbar(bath, ax=ax[0])


slope = ax[1].imshow(slopes)
ax[1].set_title("slopes")
f.colorbar(slope, ax=ax[1])

In [ ]:
bath_A_1km.values[0,:,:].shape

In [ ]:
test_array = np.array(np.arange(0,81,1).reshape(9,9))

In [ ]:
# daily_climate_1_12_padded_1 = ds_man.get_dataset("daily_climate_1_12_padded_1")
# daily_climate_1_12_padded_1_gt = ds_man.get_dataset("daily_climate_1_12_padded_1_gt")


In [ ]:
dask_climate = xa.open_dataset("lustre_scratch/datasets/global_ocean_reanalysis/daily_means/daily_climate_1_12_padded_1.nc", chunks={"time":100})
dask_climate

In [ ]:
dask_gt = spatial_data.generate_and_add_gt_to_xa_d(dask_climate,ds_man.get_dataset("monthly_climate_1_12")["coral_algae_1-12_degree"])
dask_gt

In [ ]:
dask_all = add_data_to_chunk(bath_A_1km, dask_gt).transpose("latitude", "longitude", ...)
dask_all

In [ ]:
# test = xa.open_dataset("lustre_scratch/datasets/test/padded_climate_chunks/chunk_039.nc")

f, ax = plt.subplots(figsize=[13,5])
dask_all["bathymetry_A"].isel(time=0).plot(ax=ax, cmap="Pastel2")
dask_all["coral_algae_gt"].isel(time=0).plot(ax=ax, cmap="binary", alpha=0.4)
dask_all["bottomT"].isel(time=0).plot(ax=ax, alpha=0.4)

ax.set_aspect("equal")

In [ ]:
batches = xbatcher.BatchGenerator(dask_all, 
    input_dims={"latitude": 128, "longitude": 128}, 
    input_overlap={"latitude": 4, "longitude": 4},
    )

In [ ]:
batch5 = batches[5]
# len(batches)

In [ ]:
batch5

In [ ]:
# batch5.to_netcdf("lustre_scratch/datasets/test/batches/test_batch.ncf")
from dask.distributed import Client
client = Client('dask-scheduler-address:8786')


In [ ]:
# get spatial chunk from dask xarray
chunk = dask_all.isel(latitude=slice(0,128),longitude=slice(0,128)).compute()
# process for ml

In [ ]:
# test = xa.open_dataset("lustre_scratch/datasets/test/padded_climate_chunks/chunk_039.nc")

f, ax = plt.subplots(figsize=[13,5])
chunk["coral_algae_gt"].isel(time=0).plot(ax=ax, cmap="binary", alpha=1)
chunk["bathymetry_A"].isel(time=0).plot(ax=ax, cmap="Pastel2", alpha=0.3)

chunk["bottomT"].isel(time=0).plot(ax=ax, alpha=0.2)

ax.set_aspect("equal")

In [ ]:
test_lat_lims, test_lon_lims = spatial_data.xarray_coord_limits(test,"latitude"), spatial_data.xarray_coord_limits(test,"longitude")
daily_climate_1_12_padded_1_gt["coral_algae_gt"].isel(time=0).sel(
    latitude=slice(test_lat_lims[0],test_lat_lims[1]), longitude=slice(test_lon_lims[0],test_lon_lims[1])
    ).plot(cmap="binary")

In [ ]:
test["coral_algae_gt"].isel(time=0).values

In [ ]:
daily_climate_1_12_padded_1

In [ ]:
merged = xa.open_dataset("lustre_scratch/datasets/test/padded_climate_chunks/chunk_000.nc")
merged

In [ ]:
merged["coral_algae_gt"].isel(time=0).plot()
# merged["bathymetry_A"].plot()

In [ ]:
chunk_coord_pairs, _ = nc_chunk_files("lustre_scratch/datasets/test", bath_A_1km.isel(band=0))

In [ ]:
da = bath_A_1km
_, ax, _ = spatial_plots.plot_DEM(da, "Bathymetry showing chunked data, colorcoded by number", orient_colorbar="vertical")

cmap = plt.cm.get_cmap('viridis')
for i, coord in enumerate(chunk_coord_pairs):
    color = cmap(i / len(chunk_coord_pairs))

    xy = spatial_data.index_to_coord(da, coord[0])
    height, width = spatial_data.delta_index_to_distance(da, coord[1], coord[0])
    rect = patches.Rectangle(xy, width, height, linewidth=1, edgecolor='r', facecolor=color, alpha=0.8)
    ax.add_patch(rect)

cbar2 = plt.colorbar(plt.cm.ScalarMappable(cmap=cmap), ax=ax, orientation='vertical', label="chunk number")
cbar2.set_ticklabels(np.arange(0, len(chunk_coord_pairs), 50))

plt.show()

In [ ]:
test_chunk

In [ ]:
lat_lims

In [ ]:
restricted_bath = bath_A_1km.sel({
    "latitude": slice(lat_lims[1], lat_lims[0]),
    "longitude": slice(lon_lims[0], lon_lims[1])
})
restricted_bath

In [ ]:
# attempt downsampling climate in each square
test_chunk = xa.open_dataset("lustre_scratch/datasets/test/chunk_000.nc")

lat_lims = spatial_data.xarray_coord_limits(test_chunk, "latitude")
lon_lims = spatial_data.xarray_coord_limits(test_chunk, "longitude") 

bath_res_climate_test_chunk = test_chunk.interp_like(restricted_bath, method="nearest")
bath_res_climate_test_chunk
# save as individual nc file
# set up tf dataloader: load each nc file and take batches from it

In [ ]:
bath_res_climate_test_chunk["bathymetry_A"].plot()

In [ ]:
interped = test_chunk.interp(latitude=restricted_bath["latitude"], longitude=restricted_bath["longitude"], method="nearest")
interped["bathymetry_A"].plot()

In [ ]:
# Reindex the ds_9km dataset to match the coordinates of restricted_bath
ds_9km_reindexed = test_chunk.reindex(latitude=restricted_bath.latitude, longitude=restricted_bath.longitude, method="nearest")

# Repeat the values of ds_9km for each 30 meter cell
# ds_9km_repeated = ds_9km_reindexed.repeat(restricted_bath.dims["latitude"], axis=0).repeat(restricted_bath.dims["longitude"], axis=1)

# Expand the dimensions of ds_9km to match the shape of restricted_bath
expanded_ds_9km = ds_9km_reindexed.expand_dims(latitude=restricted_bath.dims["latitude"], longitude=restricted_bath.dims["longitude"])

# Repeat the values of ds_9km for each 30 meter cell using broadcasting
ds_9km_repeated = expanded_ds_9km.broadcast_like(restricted_bath)

# Combine the datasets into a new dataset
combined_ds = xr.concat([restricted_bath, ds_9km_repeated], dim="variable")

In [ ]:
test_chunk

In [ ]:
bath_1_12_degree = spatial_data.upsample_xarray_to_target(bath_A, 1/12)

bath_1_12_degree.values[0,:,:].shape

In [ ]:
min_max_index_extreme_values(bath_1_12_degree.sel(latitude=slice(-12,-15)))

In [ ]:
bath_1_12_degree

In [ ]:
monthly_climate.sel(latitude=slice(-10,-17.05), longitude=slice(141.95,147.05))

In [ ]:
no_band_bath = bath_1_12_degree.isel(band=0)

# downsample climate data to 1km
monthly_climate = ds_man.get_dataset("monthly_climate_1_12")

# get limits of bathymetry
lat_lims = spatial_data.xarray_coord_limits(bath_1_12_degree, "latitude")
lon_lims = spatial_data.xarray_coord_limits(bath_1_12_degree, "longitude")

restricted_monthly_climate = monthly_climate.sel(latitude=slice(-10,-17), longitude=slice(142,147))


# padded_restricted_monthly_climate = spatial_data.buffer_nans(restricted_monthly_climate, size=1
km_monthly = restricted_monthly_climate.interp_like(bath_1_12_degree, method="linear")

coral_climate_1km = xa.combine_by_coords([km_monthly.drop("spatial_ref"),no_band_bath], coords=["time", "latitude", "longitude"])
(coral_climate_1km,) = xa.broadcast(coral_climate_1km)
coral_climate_1km

In [ ]:
restricted_monthly_climate

In [ ]:
# TODO: fix ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (13,) + inhomogeneous part.

# train_comb_Xs, train_comb_ys, train_comb_subsample, train_comb_lat_lons_vals_dict = generate_patch(xa_ds=coral_climate_1km, lat_lon_starts=(-10,142), coord_range=(-6,6))
# test_comb_Xs, test_comb_ys, test_comb_subsample, test_comb_lat_lons_vals_dict = generate_patch(xa_ds=coral_climate_1km, lat_lon_starts=(-16,148), coord_range=(-6,6))

In [ ]:
print("all_Xs_onehot shape: ", all_Xs_onehot.shape)
print("all_ys_onehot shape: ", all_ys_onehot.shape)

In [ ]:
# TODO: normalise along variable axes

In [ ]:
all_Xs, all_lat_lon_dict = sample_spatial_batch(xa_coral_climate_1_12, lat_lon_starts=(-8,140), coord_range=(-20,13))
all_Xs, all_lat_lon_dict = subsample_to_array(xa_coral_climate_1_12, lat_lon_starts=(-8,140), coord_range=((-20,13)))
all_Xs = naive_X_nan_replacement(all_Xs)
all_ys, _ = subsample_to_array(xa_coral_climate_1_12, lat_lon_starts=(-10,142), coord_range=(-7,5), variables = ["coral_algae_1-12_degree"])
all_ys = naive_y_nan_replacement(all_ys)
all_ys = all_ys[:,:,0]

In [ ]:
print("Xs shape: ", Xs.shape)
print("ys shape: ", ys.shape)

In [ ]:
Xs, ys, all_subsample, all_lat_lons_vals_dict = generate_patch(xa_coral_climate_1_12, lat_lon_starts=(-10,142), coord_range=(-7,5))
patch1_Xs, patch1_ys, patch1_subsample, patch1_lat_lons_vals_dict = generate_patch(xa_coral_climate_1_12, (-10,142), (-7,5))
patch2_Xs, patch2_ys, patch2_subsample, patch2_lat_lons_vals_dict = generate_patch(xa_coral_climate_1_12, (-17,147), (-7,5))
patch3_Xs, patch3_ys, patch3_subsample, patch3_lat_lons_vals_dict = generate_patch(xa_coral_climate_1_12, (-16,146), (-7,5))



In [ ]:
dailies_combined = xa.open_dataset("/Volumes/MRes Drive/global_ocean_reanalysis/daily/dailies_combined.nc").isel(depth=0)
monthly = xa.open_dataset("/Volumes/MRes Drive/global_ocean_reanalysis/monthly_means/baseline_area/coral_climate_1_12.nc")
coral_gt, lat_lon_A_vals_dict = spatial_data.sample_spatial_batch(
       monthly["coral_algae_1-12_degree"], lat_lon_starts=(-10,141.95), coord_range=(-7.01,5.11))

dailies_combined["coral_algae_1-12_degree"] = coral_gt.isel(time=0)
(dailies_combined,) = xa.broadcast(dailies_combined)

In [ ]:
coral_gt.isel(time=0)

In [ ]:
dailies_combined

In [ ]:
dailies_combined["coral_algae_1-12_degree"] = coral_gt.isel(time=0)
(dailies_combined,) = xa.broadcast(dailies_combined)

In [ ]:
dailies_combined

In [ ]:
# really slow, even when ds isn't particularly large (5185, 9863, 8): probably due to ds.to_array().values
# in xa_d_to_np_array
(daily_Xs, daily_ys), daily_subsample, daily_lat_lons_vals_dict = spatial_data.generate_patch(
    xa_ds=daily_climate_1_12, lat_lon_starts=(-17,142), coord_range=(7,5))

In [ ]:
print("Xs shape: ", daily_Xs.shape)
print("ys shape: ", daily_ys.shape)

In [ ]:
X_train = daily_Xs[:]
y_train = daily_ys[:]

wandb.init(
    project="coralshift",
    entity="orlando-code",
    settings=wandb.Settings(start_method="fork")
    # config={    }
    )

# initialize optimiser: will need hyperparameter scan for learning rate and others
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
optimizer = tf.keras.optimizers.Adam(3e-4)

# X = ds_man.get_dataset("monthly_climate_1_12_X_np")
# y = ds_man.get_dataset("monthly_climate_1_12_y_np")
# # check that untrained model runs (should output array of non-nan values)
# # why values change?
# # g_model(X[:32])

# X_train, X_test, y_train, y_test = model_selection.train_test_split(
#     X, y, test_size=0.2, random_state=42)

# X_train, X_test, y_train, y_test = model_selection.train_test_split(
#     sub_X, sub_y, test_size=0.2, random_state=42)

# Define Gated Recurrent Unit model class in TensorFlow
class gru_model(tf.keras.Model):
    # initialise class instance to define layers of the model
    def __init__(self, rnn_units: list[int], num_layers: int, 
        # dff: int
        ):
        """Sets up a GRU model architecture with multiple layers and dense layers for mapping the outputs of the GRU 
        layers to a desired output shape

        Parameters
        ----------
        rnn_units (list[int]): list containing the number of neurons to use in each layer
        num_layers (int): number of layers in GRU model
        """
        super(gru_model, self).__init__()   # initialise GRU model as subclass of tf.keras.Model
        # store values for later use
        self.num_layers = num_layers    # number of layers in GRU model
        self.rnn_units = rnn_units
        # self.dff = dff
        # define model layers: creating new `tf.keras.layers.GRU` layer for each iteration
        self.grus = [tf.keras.layers.GRU(rnn_units[i],  # number (integer) of rnn units/neurons to use in each model layer
                                   return_sequences=True,   # return full sequence of outputs for each timestep
                                   return_state=True) for i in range(num_layers)] # return last hidden state of RNN at end of sequence
        
        # dense layers are linear mappings of RNN layer outputs to desired output shape
        # self.w1 = tf.keras.layers.Dense(dff) # 10 units
        self.w1 = tf.keras.layers.Dense(10) # 10 units

        self.w2 = tf.keras.layers.Dense(1)  # 1 unit (dimension 1 required before final sigmoid function)
        # self.A = tf.keras.layers.Dense(30)
        # self.B = tf.keras.layers.Dense(dff)



    def call(self, inputs: np.ndarray, training: bool=False):
        """Processes an input sequence of data through several layers of GRU cells, followed by a couple of
        fully-connected dense layers, and outputs the probability of an event happening.
        
        Parameters
        ----------
        inputs (np.ndarray): input tensor of shape (batch_size, seq_length, features)
            batch_size - defines the size of the sample drawn from datapoints
            seq_length - number of timesteps in sequence
            features - number of features associated with each datapoint
        training (bool, defaults to False): True if model is in training, False if in inference mode

        Returns
        -------
        target: probability of an event occuring, with shape (batch_size, 1)
        """
        # input shape: (batch_size, seq_length, features)
       
        assert self.num_layers == len(self.rnn_units)

        # check that input tensor has correct shape
        if (len(inputs.shape) != 3):
            print(f"Incorrect shape of input tensor. Expected 3D array. Recieved {len(inputs.shape)}D array.")

        # print('input dim ({}, {}, {})'.format(inputs.shape[0], inputs.shape[1], inputs.shape[2]))
        # whole_seq, static_input = inputs
        whole_seq = inputs


        # iteratively passes input tensor to GRU layers, overwritting preceding sequence 'whole_seq'
        for layer_num in range(self.num_layers):
            whole_seq, final_s = self.grus[layer_num](whole_seq, training=training)

        # adding extra layers
        # static = self.B(tf.nn.gelu(self.A(static_input)))
        # target = self.w1(final_s)  + static # final hidden state of last layer used as input to fully connected dense layers...
        target = self.w1(final_s)   # final hidden state of last layer used as input to fully connected dense layers...

        target = tf.nn.relu(target) # via ReLU activation function
        target = self.w2(target)    # final hidden layer must have dimension 1 
        
        # obtain a probability value between 0 and 1
        target = tf.nn.sigmoid(target)
        
        return target


# initialise GRU model with 500 hidden layers, one GRU unit per layer 
g_model = gru_model([100], 1) # N.B. [x] is number of hidden layers in GRU network


def negative_log_likelihood(y: np.ndarray, y_pred: np.ndarray, class_weights: np.ndarray = None) -> float:
    """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
    incorporating class weights.

    Parameters
    ----------
    y (np.ndarray): true binary labels, where 0 represents the negative class
    y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
    class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

    Returns
    -------
    float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
    incorporating class weights if provided
    """
    bce = tf.keras.losses.BinaryCrossentropy()  

    if class_weights is not None:
        sample_weights = tf.gather(class_weights, np.asarray(y,dtype=np.int32))
        return bce(y, y_pred, sample_weight=sample_weights)

    return bce(y, y_pred)


def training_batches(X: np.ndarray, y: np.ndarray, batch_num: int, batch_size: int=32):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size

    X_batch = X[start_idx:end_idx]
    y_batch = y[start_idx:end_idx]
    
    return X_batch, y_batch

# https://stackoverflow.com/questions/52357542/attributeerror-tensor-object-has-no-attribute-numpy
# should aim to delete the following to speed up training: but can't figure out a way to make wandb reporting work
# without it
tf.config.run_functions_eagerly(True)

def build_graph():
    
    # compile function as graph using tf's autograph feature: leads to faster execution times, at expense of limitations
    # to Python objects/certain control flow structures (somewhat relaxed by experimental_relax_shapes)
    @tf.function(experimental_relax_shapes=True)
    def train_step(gru: tf.keras.Model, optimizer: tf.keras.optimizers.Optimizer, X: np.ndarray, y: np.ndarray, 
        training: bool=True, class_weights=class_weights, batch_num:int=None, batch_size: int=None) -> tuple[np.ndarray, float]:
        """Train model using input `X` and target data `y` by computing gradients of the loss (via 
        negative_log_likelihood)
        
        Parameters
        ----------
        y (np.ndarray): true binary labels, where 0 represents the negative class
        y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)

        Returns
        -------
        float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred'
        """
        if training:
            num_samples = X.shape[0]
            num_batches = num_samples // batch_size
            # num_batches = batch_num
            total_epoch_loss = 0.0
            for batch_num in tqdm(range(num_batches), desc="batches", position=0, leave=True):
                X_batch, y_batch = training_batches(X, y, batch_num=batch_num, batch_size=batch_size)

                with tf.GradientTape(persistent=True) as tape:
                    y_pred = gru(X_batch, training) 
                    xent = negative_log_likelihood(y_batch, y_pred, class_weights)
                    # y_pred = gru(X, training) # TO DELETE
                    # xent = negative_log_likelihood(y, y_pred)
                
                gradients = tape.gradient(xent, gru.trainable_variables)
                optimizer.apply_gradients(zip(gradients, gru.trainable_variables))
                # print("xent", xent.numpy())
                # print("total_epoch_loss", total_epoch_loss)
                total_epoch_loss += xent
                # learning rate?
                wandb.log({"batch": batch_num, "loss": xent, "total_epoch_loss": total_epoch_loss})

            average_loss = total_epoch_loss / num_batches
            # return predicted output values and total loss value
            return y_pred, xent, total_epoch_loss

    # set default float type
    tf.keras.backend.set_floatx('float32')
    # TODO: this isn't assigned... What should it return otherwise?
    return train_step



class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=np.reshape(y_train,-1))

with tf.device("/GPU:0"):
    num_epochs = 10
    # will update so that subsamples are fed in from which batches are taken: will require recomputation
    # of class_weight for each subsample
    num_batches = 2
    batch_size = 128
    tr_step = build_graph()
    for epoch in tqdm(range(num_epochs), desc= " epochs", position = 0, leave=True):
        y_pred, xent, total_epoch_loss = tr_step(
            g_model, optimizer, X_train[:], y_train[:], class_weights=class_weights, 
            batch_size=batch_size, batch_num=num_batches, training=True)
        wandb.log({"epoch": epoch, "epoch_loss": total_epoch_loss})

wandb.finish()

# check (with prints) that wandb is functioning
# check against known timeseries task for correct implementation
# find timeseries which get bad loss and debug why
# log best loss which can be logged: save weights (and do a inference plot with best weights)

In [ ]:
(daily_subsample_Xs, daily_subsample_ys), daily_subsample, daily_subsample_lat_lon_vals_dict = spatial_data.generate_patch(xa_ds=daily_climate_1_12, lat_lon_starts=(-17,144), coord_range=(3,3))

# sample, ll_dict = spatial_data.sample_spatial_batch(xa_ds=daily_climate_1_12, lat_lon_starts=(-17,145), coord_range=(-2,2))

In [ ]:
arr=[0,1,2,3,4,5]
for i in range(0,3):
    print(arr[:-i or None])

In [ ]:
daily_subsample_Xs.shape

In [ ]:
### train/predict
# if training:
    # assign epoch output to dataset
# if testing:
# patch1_pred = g_model(patch1_Xs, training=False)
# patch2_pred = g_model(patch2_Xs, training=False)
# patch3_pred = g_model(patch3_Xs, training=False)
pred = g_model(daily_subsample_Xs, training=False)

# all_predicted = g_model(all_Xs, training=False)
    # assign output predictions to dataset

In [ ]:
patch = spatial_data.reformat_prediction(daily_climate_1_12, daily_subsample, pred, daily_subsample_lat_lon_vals_dict)
spatial_plots.plot_var(patch["output"])


In [ ]:
patch_pred_xa_ds_onehot_lim = reformat_prediction(xa_coral_climate_1_12_working, test_onehot_subsample, pred, test_onehot_lat_lons_vals_dict)

mask = patch_pred_xa_ds_onehot_lim["output"] > 0.6

spatial_plots.plot_var(mask)

In [ ]:
patch_pred_xa_ds_onehot_lim = reformat_prediction(xa_coral_climate_1_12_working, test_onehot_subsample, pred, test_onehot_lat_lons_vals_dict)

spatial_plots.plot_var(patch_pred_xa_ds_onehot_lim["output"])



In [ ]:
patch_Xs_onehot, patch_ys_onehot, patch_subsample_onehot, patch_lat_lons_vals_dict_onehot = generate_patch(xa_coral_climate_1_12, (-10,142), (-7,5))

patch_pred_xa_ds_onehot = reformat_prediction(xa_coral_climate_1_12_working, patch_subsample_onehot, pred, patch_lat_lons_vals_dict_onehot)

spatial_plots.plot_var(patch_pred_xa_ds_onehot["output"])


In [ ]:
np.array_equal(patch1_pred_xa_ds["output"].values,patch2_pred_xa_ds["output"])

In [ ]:
# patch1_pred_xa_ds = reformat_prediction(xa_coral_climate_1_12_working, patch1_subsample, patch1_pred, patch1_lat_lons_vals_dict)
# patch2_pred_xa_ds = reformat_prediction(xa_coral_climate_1_12_working, patch2_subsample, patch2_pred, patch2_lat_lons_vals_dict)
# patch3_pred_xa_ds = reformat_prediction(xa_coral_climate_1_12_working, patch3_subsample, patch3_pred, patch3_lat_lons_vals_dict)


f, a0 = spatial_plots.plot_var_at_time(xa_coral_climate_1_12["coral_algae_1-12_degree"], "2020-12-16")
# # visualise result
# spatial_plots.plot_var(patch1_pred_xa_ds["output"])
# spatial_plots.plot_var(patch2_pred_xa_ds["output"])
# spatial_plots.plot_var(patch3_pred_xa_ds["output"])

# pred_xa_ds["coral_algae_1-12_degree"].isel(time=-1).plot(ax=ax[0])
# pred_xa_ds["output"].plot(ax=ax[1])


In [ ]:
xa_coral_climate_1_12["bottomT"].isel(time=-1).plot()

# Adding in bathymetry

In [ ]:
# downsample climate data to 1km
monthly_climate = ds_man.get_dataset("monthly_climate_1_12")

# get limits of bathymetry
lat_lims = spatial_data.xarray_coord_limits(coarsened_bath_A, "latitude")
lon_lims = spatial_data.xarray_coord_limits(coarsened_bath_A, "longitude")

restricted_monthly_climate = monthly_climate.sel(latitude=slice(-10,-17), longitude=slice(142,147))


# padded_restricted_monthly_climate = spatial_data.buffer_nans(restricted_monthly_climate, size=1

In [ ]:
buffer_size = 3
exclude_vars = ["spatial_ref", "coral_algae_1-12_degree", "siconc", "usi", "vsi", "sithick"]
buffered_ds = spatially_buffer_timeseries(monthly_climate, buffer_size=buffer_size, exclude_vars=exclude_vars)

buffered_ds.to_netcdf(
    ds_man.get_location() / f"global_ocean_reanalysis/monthly_means/monthly_climate_{buffer_size}_buffer.nc")

In [ ]:
buffered_ds

In [ ]:
f,a = plt.subplots(1,2, figsize=[10,5])
monthly_climate["mlotst"].isel(time=99).plot(ax=a[0], cmap="jet")
buffer_attempt["mlotst"].isel(time=99).plot(ax=a[1],cmap="jet")

In [ ]:
coral_climate_1km

In [ ]:
buffer_attempt.equals(monthly_climate.isel(time=slice(0,2)))

In [ ]:
buffer_attempt.isel(time=1)["mlotst"].plot()

In [ ]:
coarsened_bath_A.isel(band=0)

In [ ]:
coral_climate_1km["bathymetry_A"]

In [ ]:
monthly_climate

In [ ]:
f,a = plt.subplots(1,2, figsize=[10,5])
coral_climate_1km.isel(time=-1)["bathymetry_A"].plot(ax=a[0], vmin=-100, vmax=0)
coral_climate_1km.isel(time=-1)["mlotst"].plot(ax=a[1])

In [ ]:
# attempt.isel(time=-1)["mlotst"].plot()
eg_data = buffer_attempt.isel(time=-1)["mlotst"]

spatial_plots.plot_DEM(eg_data, f" DEM upsampled to {target_resolution} meters", 
    landmask=False, vmin=np.nanmin(eg_data.values), vmax=np.nanmax(eg_data.values), cmap="jet")

In [ ]:
# TESTING

# for longitude in array
# get 
sub_X.shape

In [ ]:
X.shape

In [ ]:
wandb.finish()

In [ ]:
# TODO: optionally replace batching with spatial batching

### Test GRU functions

In [ ]:
print(tf.config.list_physical_devices())
!nvidia-smi

In [ ]:
with tf.device("/GPU:0"):
    num_epochs = 5
    num_batches = 100
    tr_step = build_graph()
    for epoch in tqdm(range(num_epochs), desc= " epochs", position = 0):
        y_pred, average_loss = tr_step(g_model, optimizer, X_train[:1000], y_train[:1000], batch_size=32, training=True)
        
        
        
        # for batch in range(num_batches):
        #     array, y  = batcher_fun(X, 32, 276 
        #     #training = True)# shapes: (batch_s, seq_l, features), (batch_s, 1)
        #     )
        #     y_pred, xent = tr_step(g_model, optimizer, X[:32], y, training=True)
            
        #  ## validation set 
        #  ## test_set 

In [ ]:
# negative_log_likelihood(y_test,g_model(X_test))
y_test=y_test[:1000]
y_pred = g_model(X[:1000])

In [ ]:
plt.scatter(y_test,y_pred.numpy())

In [ ]:
# made unnecssary due to isel indexing
# def pixels_to_coord_diff(xa_ds: xa.Dataset | xa.DataArray, window_dim: int, coord: str) -> list[float, float]:
#     return float(window_dim * np.diff(spatial_data.min_max_of_coords(xa_ds, coord)) / len(list(xa_coral_climate_1_12[coord])))

In [ ]:
subsample["bottomT"].isel(time=-1).plot()

In [ ]:
sample_spatial_batch(xa_coral_climate_1_12,lat_lon_starts=(-16,144), coord_range=(-4,2))["coral_algae_1-12_degree"]

In [ ]:
xa_coral_climate_1_12_working = xa_coral_climate_1_12

In [ ]:
# [lat_lon_vals_dict.items() for key in ["latitude", "longitude"]]
{key: lat_lon_vals_dict[key] for key in ["latitude", "longitude"]}

In [ ]:
sub_y_nans.shape

In [ ]:
(list(subsample.data_vars))

In [ ]:
# TODO: fix boolean indexing
# # For now, shallowest depth is taken (0.45)
# # TODO: process this and export it to new file since takes a while to run
# ds_man.add_dataset(
#     "daily_climate_1_12_X_np", filter_out_nans(
#         spatial_data.xa_ds_to_3d_numpy(ds_man.get_dataset("daily_climate_1_12").isel(depth=0)), ds_man.get_dataset("daily_climate_1_12_y_np"))[0]
# )

In [ ]:
# TODO: put this merge into data processing pipeline
# merge daily mean files
# var_daily_dir = Path("lustre_scratch/datasets/global_ocean_reanalysis/daily_means")
# save_combined_dailies_path = Path("lustre_scratch/datasets/global_ocean_reanalysis/daily_means/dailies_combined.nc")
# daily_file_paths = file_ops.return_list_filepaths(var_daily_dir, ".nc")
# combined_dailies = xa.open_mfdataset(daily_file_paths)
# combined_dailies.to_netcdf(save_combined_dailies_path)

In [ ]:
# # create 3D array from xarray dataset variables. Shape: (num_samples, num_parameters, sequence_len)
# X_with_nans = spatial_data.xa_ds_to_3d_numpy(xa_coral_climate_1_12_features)
# print(f'X_with_nans shape (num_samples: {X_with_nans.shape[0]}, total num_parameters (includes nans parameters): {X_with_nans.shape[1]}, sequence_len: {X_with_nans.shape[2]})')

# for i, param in enumerate(xa_coral_climate_1_12_features.data_vars):
#     print(f"{i}: {param}")

#### Remove observations for which there are nan values

99% sure these are are just gridcells containing land. Would be a good thing to investigate, however.

In [ ]:
# X = X_with_nans
# # problem, probably with sea ice features

In [ ]:
# # filter out columns that contain entirely NaN values
# col_mask = ~np.all(np.isnan(X), axis=(0,2)) # boolean mask indicating which columns to keep
# masked_cols = X[:, col_mask, :] # keep only the columns that don't contain entirely NaN values
# print("masked_cols shape:", masked_cols.shape)

In [ ]:
# # filter out all rows which contain any NaN values
# row_mask = ~np.any(np.isnan(masked_cols), axis=1) # boolean mask indicating which rows to keep
# masked_cols_rows = masked_cols[row_mask[:,0], :, :] # keep only the rows that don't contain any NaN values
# masked_cols_rows.shape

In [ ]:
# # filter out all depths which contain any NaN values
# depth_mask = ~np.any(np.isnan(masked_cols_rows), axis=(0,1)) # boolean mask indicating which depths to keep
# X = masked_cols_rows[:, :, depth_mask] # keep only the depths that don't contain any NaN values
# X = np.swapaxes(X, 1, 2)
# print(f"X shape: {X.shape}")

In [ ]:
# # create target from coral ground truth. Shape: (num_samples, 1)
# # TODO: not sure if this is shuffling the values when reshaping
# y_with_nans = np.array(xa_coral_climate_1_12["coral_algae_1-12_degree"].sel(
#     time=xa_coral_climate_1_12.time[-1])).reshape(-1, 1)
# # remove ys with nan values in other variables
# y = y_with_nans[row_mask[:,0]]

# print(f"y_with_nans shape: {y_with_nans.shape}")
# print(f"y shape: {y.shape}")

In [ ]:
# X, y = filter_out_nans(X_with_nans, np.array(xa_coral_climate_1_12["coral_algae_1-12_degree"].isel(time=-1)).reshape(-1, 1))
# print(f"X shape: {X.shape}")
# print(f"y shape: {y.shape}")

In [ ]:
# X.shape

In [ ]:
# sub_X = np.moveaxis(np.array(test_array), 2, 1)
sub_X.shape

In [ ]:
col_mask = ~np.all(np.isnan(test_array), axis=(0,2))
sub_X = test_array[:, col_mask, :]

In [ ]:
sub_X.shape

In [ ]:
xa_coral_climate_1_12_features = ds_man.get_dataset("monthly_climate_features")
xa_coral_climate_1_12 = ds_man.get_dataset("monthly_climate_1_12")

In [ ]:
Xs, lat_lon_dict = subsample_to_array(xa_coral_climate_1_12, lat_lon_starts=(-10,142), coord_range=(-7,5))
Xs_nonans = naive_X_nan_replacement(Xs)
ys, _ = subsample_to_array(xa_coral_climate_1_12, lat_lon_starts=(-10,142), coord_range=(-7,5), variables = ["coral_algae_1-12_degree"])
ys = ys[:,:,0]

In [ ]:
ys.shape

In [ ]:
# sample subset
# subsample, lat_lon_vals_dict = sample_spatial_batch(xa_coral_climate_1_12_features,lat_lon_starts=(-12,144), coord_range=(-4,2))
# subsample, lat_lon_vals_dict = sample_spatial_batch(xa_coral_climate_1_12_features,lat_lon_starts=(-10,142), coord_range=(-7,5))
# convert to ndarray
# # test_array = spatial_data.xa_ds_to_3d_numpy(subsample)
# # subsample_all, _ = sample_spatial_batch(xa_coral_climate_1_12,lat_lon_starts=(-16,144), coord_range=(-4,2))
# subsample_all, _ = sample_spatial_batch(xa_coral_climate_1_12,lat_lon_starts=(-10,142), coord_range=(-7,5))
# sub_y_nans = (np.array(subsample_all["coral_algae_1-12_degree"].isel(time=-1))).reshape(-1, 1)
# # remove nans
# #sub_X, sub_y = filter_out_nans(test_array, sub_y_nans)
# # testing, so replace nans with -1
# # filter out columns that contain entirely NaN values
# # col_mask = ~np.all(np.isnan(test_array), axis=(0,2)) # boolean mask indicating which columns to keep
# # sub_X = test_array[:, col_mask, :] # keep only the columns that don't contain entirely NaN values

# # sub_X = np.moveaxis(np.array(sub_X), 2, 1)
# sub_y = sub_y_nans
# # sub_X[np.isnan(sub_X)] = -10000
# sub_y[np.isnan(sub_y)] = -10000




In [ ]:
def subset_to_dataset_var(xa_ds: xa.Dataset | xa.DataArray, subset_vals: np.ndarray, dims: list=['latitude', 'longitude', "time"]):

In [ ]:
predicted

In [ ]:
test_x_train = np.append(100*np.ones((50,50,5)), 1*np.ones((50,50,5)), 0)
test_y_train = np.append(np.ones(50,), np.zeros(50,))


print("test_x_train:", test_x_train.shape)
# print("x_test:", x_test.shape)
print("test_y_train:", test_y_train.shape)
# print("y_test:", y_test.shape)


In [ ]:
array = np.random.normal(size = (32, 20, 1))    # shape: (num_samples, sequence_length, num_features)
y_dud = np.random.choice([0, 1], size = 32)
print("array shape:", array.shape)
print("y_dud shape:", y_dud.shape)

x_train, y_train = X[:500], y[:500].reshape(500,)
# x_test, y_test = X[5000:6000], y[5000:6000].reshape((1000,))

print("x_train:", x_train.shape)
# print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
# print("y_test:", y_test.shape)


In [ ]:
plt.plot(g_model(test_x_train[:],training=False).numpy())

In [ ]:
predicted = g_model(X[:5610],training=False)

In [ ]:
X.shape

In [ ]:
xa_coral_climate_1_12["coral_algae_1-12_degree"].isel(time=-1)

In [ ]:
30*187

In [ ]:
fig, ax = plt.subplots()
out = ax.imshow(predicted.numpy().reshape(30,187))
fig.colorbar(out, ax=ax)

In [ ]:
np.sqrt(500)

In [ ]:
xa_coral_climate_1_12_features

In [ ]:
xa_coral_climate_1_12["coral_algae_1-12_degree"].isel(time=-1)

In [ ]:
fig, ax = plt.subplots()
out = ax.imshow(y_pred.numpy().reshape(20,25))
fig.colorbar(out, ax=ax)

In [ ]:
sum((y_pred > 0.5).numpy())

In [ ]:
# check log likelihood is computable
negative_log_likelihood(y[:32], g_model(X[:32]))

## Train and test GRU

In [ ]:
# define batcher function (by space and time)

In [ ]:
def batcher_fun(X, y, batch_size, seq_len):
    """
    A function to prepare the data for training the LSTM.
    
    :param data: The input data to the LSTM.
    :param batch_size: The number of samples in each batch.
    :param seq_len: The sequence length of each sample.
    
    :return: A tuple of (batch_x, batch_y), where batch_x is a numpy array of shape (batch_size, seq_len, num_features) 
             and batch_y is a numpy array of shape (batch_size, num_classes).
    """
    num_samples = len(data)
    num_batches = int(num_samples / batch_size)
    num_features = spatial_data.shape[1]
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        batch_x = np.zeros((batch_size, seq_len, num_features))
        batch_y = np.zeros((batch_size, 1))
        
        for j in range(start_idx, end_idx):
            sample = data[j]
            X = sample[:-1]
            y = y[]
            
            batch_x[j - start_idx] = x.reshape((seq_len, num_features))
            batch_y[j - start_idx, y] = 1
            
        yield batch_x, batch_y


In [ ]:
with tf.device("/CPU:0"):
    num_epochs = 1
    num_batches = 100
    tr_step = build_graph()
    for epoch in range(num_epochs):
        for batch in range(num_batches):
            array, y  = batcher_fun(X, 32, 276 
            #training = True)# shapes: (batch_s, seq_l, features), (batch_s, 1)
            )
            y_pred, xent = tr_step(g_model, optimizer, X[:32], y, training=True)
            
         ## validation set 
         ## test_set 

In [ ]:
y_pred

# Copypasta

[Source](https://github.com/christianversloot/machine-learning-articles/blob/main/build-an-lstm-model-with-tensorflow-and-keras.md)

In [ ]:
X.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam # https://stackoverflow.com/questions/75356826/attributeerror-adam-object-has-no-attribute-get-updates
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model configuration
additional_metrics = ["accuracy"]
# batch_size = 128
batch_size = 32
# embedding_output_dims = 15
# embedding_output_dims = 10
loss_function = BinaryCrossentropy()
# max_sequence_length = 300
max_sequence_length = 276
# num_distinct_words = 5000
# num_distinct_words = 10000
number_of_epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

# Disable eager execution
tf.compat.v1.disable_eager_execution()

In [ ]:
# Load dataset
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)
x_train, y_train = X[:5000], y[:5000].reshape((5000,))
x_test, y_test = X[5000:6000], y[5000:6000].reshape((1000,))

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

In [ ]:
# Pad all sequences: keras requires sequences of equal lengths. Should be handled in pre-processing, but here for now for security
padded_inputs = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

# (number_samples, sequence_length, num_features)
print("padded_inputs:", padded_inputs.shape)
print("padded_inputs_test:", padded_inputs_test.shape)

In [ ]:
padded_inputs = pad_sequences(x_train[:,:,0], maxlen=max_sequence_length, value = 0.0)
padded_inputs_test = pad_sequences(x_test[:,:,0], maxlen=max_sequence_length, value = 0.0)

# (number_samples, sequence_length)
print("padded_inputs:", padded_inputs.shape)
print("padded_inputs_test:", padded_inputs_test.shape)

In [ ]:
# Define the Keras model
model = Sequential()
model.add(
    Embedding(
        num_distinct_words+1, embedding_output_dims, input_length=max_sequence_length
    )
)
model.add(LSTM(10))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# Give a summary
model.summary()

In [ ]:
y_train.shape

In [ ]:
# Train the model
history = model.fit(
    padded_inputs,
    y_train,
    batch_size=batch_size,
    epochs=number_of_epochs,
    verbose=verbosity_mode,
    validation_split=validation_split,
)

# Test the model after training
test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
print(f"Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%")

In [ ]:
def plot_score_timeseries(history) -> None:
    fig, ax = plt.subplots()
    ax.plot(history.history["accuracy"])
    ax.plot(history.history["val_accuracy"])

    ax.set_title("Model accuracy against epoch")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    ax.legend(['train set', 'validation set'], loc='upper left')

plot_score_timeseries(history)

In [ ]:
model.metrics_names

# Multivariate model

[Source](https://medium.com/@canerkilinc/hands-on-multivariate-time-series-sequence-to-sequence-predictions-with-lstm-tensorflow-keras-ce86f2c0e4fa)

In [ ]:
X_toy = X[:32*10,:10,:3]
print("X_toy:", X_toy.shape)
y_toy = y[:32*10]
print("y_toy:", y_toy.shape)


In [ ]:
#import packages
import tensorflow
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model
#####################################
#Before do anything else do not forget to reset the backend for the next iteration (rerun the model)
tensorflow.keras.backend.clear_session()
#####################################
# Initialising the LSTM Model with MAE Loss-Function
batch_size = 32
epochs = 120
timesteps = 10
num_features = 3
input_1 = Input(batch_shape=(batch_size,timesteps,num_features))
#each layer is the input of the next layer
lstm_hidden_layer_1 = LSTM(10, stateful=True, return_sequences=True)(input_1)
lstm_hidden_layer_2 = LSTM(10, stateful=True, return_sequences=True)(lstm_hidden_layer_1)
output_1 = Dense(units = 1)(lstm_hidden_layer_2)
regressor_mae = Model(inputs=input_1, outputs = output_1)
#adam is fast starting off and then gets slower and more precise
#mae -> mean absolute error loss function
regressor_mae.compile(optimizer='adam', loss = 'mae')
#####################################
#Summarize and observe the layers as well as paramter configurations
regressor_mae.summary()

In [ ]:
regressor_mae.fit(
    
)